In [43]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import os

In [44]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')


In [45]:
import numpy as np
from tqdm import tqdm

In [49]:
model['maman'].shape

(300,)

def get_definitions(word, from_lang, to_lang, print_output = False):
    senseword_list = []
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    if print_output == True: print(url)
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    #print(soup.prettify())
    first_string_passed = False
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            senseword_list.append(sence_word)
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            senseword_list.append(link.text.strip())
    time.sleep(2)
    return senseword_list
    """
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        print("========================================")
        print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + word + "\w{0,2}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                print(example)
    """
 
get_definitions("согревать", "русский", "французский", print_output = False)



In [6]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_lemmas_only.csv")
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,для,"['for the bank', 'for last month', 'for one ye...","['для банка', 'за последний месяц', 'на один г..."
1,1,for,ради,['for god'],['ради бога']
2,2,for,в течение,[],[]
3,3,for,в отношении,[],[]
4,4,for,для целей,[],[]


In [46]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [47]:
from  sklearn.metrics.pairwise import cosine_similarity

In [ ]:
fr_list = ['mère', 'maman', 'mère de famille', 
           'materner', 'Vierge', 'Mother', 'parents', 'femme', 'maternel', 'fils', 'Dame', 'Marie']

In [48]:
def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
def preprocess_words(words_list, lemmatizer_func):
    preprocessed_words = []
    for word in words_list:
        lemma = lemmatizer_func(word.lower())
        #print(lemma)
        preprocessed_words.append(lemma)
    return preprocessed_words
        

In [51]:
l1 = "maman"
l2 = 'mère de famille'

def get_vectors_sum(lemm_line):
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2):
    l1_vect = get_vectors_sum(line_1)
    l2_vect = get_vectors_sum(line_2)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2)      

0.3063980830674433

In [52]:
def get_two_target_lang_lists_intersection(list_1, list_2):
    overall_intersection = []
    cos_sim_dict ={}
    for word_1 in list_1:
        for word_2 in list_2:
            if word_1 == word_2:
                #overall_intersection.append(word_1)
                cos_sim_dict[word_1] = []
    
    handled_words = []
    for word_1 in list_1:
        for word_2 in list_2:
            if word_1 != word_2 and word_1 not in handled_words and word_2 not in handled_words:
                sim = find_lines_similarity(word_1, word_2)
                if sim > 0.95:
                    if word_1 in cos_sim_dict:
                        handled_words.append(word_2)
                        cos_sim_dict[word_1].append(word_2)
                       # overall_intersection[word_1_index] += '| ' + word_2
                    elif word_2 in cos_sim_dict:
                        handled_words.append(word_1)
                        cos_sim_dict[word_2].append(word_1)
                    else:
                        cos_sim_dict[word_1] = [word_2]
                        #new_pair = word_1 + " | " + word_2
                        #overall_intersection.append(new_pair)
    #print(cos_sim_dict)
    for first_word in cos_sim_dict.keys():
        intersect_element = first_word
        for similar_word in cos_sim_dict[first_word]:
            intersect_element += " | " + similar_word
        overall_intersection.append(intersect_element)
    return overall_intersection

fr_proc_1 = ['mère', 'maman', 'mère de famille', 'materner', 
             'vierge', 'mother', 'parent', 'femme', 'maternel', 'fil', 'dame', 'marie']
fr_proc_2 = ['mère', 'maman', 'putain', 'merde', 'quoi', 'enfoiré', 
             'est-ce', 'dieu', 'bordel', 'connard', 'enculé']
get_two_target_lang_lists_intersection(fr_proc_1, fr_proc_2)      
            

['mère', 'maman']

In [65]:
def get_definitions(word, lang):
    if lang == "ru":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_ru_fr/", word + ".json")
    elif lang == "en":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_en_fr_whole_list/", word + ".json")
    local_word = []
    with open(word_path, "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                local_word.append(translation['text'])
    return local_word

In [67]:
def get_cross_translate_yand(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_list = get_definitions(word_lang_1, lang_1)
    lang_1_processed = preprocess_words(lang_1_to_target_words_list, lemmatizer_func)
    print(lang_1_processed)
    lang_2_to_target_words_list = get_definitions(word_lang_2, lang_2)
    lang_2_processed = preprocess_words(lang_2_to_target_words_list, lemmatizer_func)
    print(lang_2_processed)
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_processed, lang_2_processed)
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate_yand("warm", "en", "согревать","ru", "fr", fr_lemmatize)


['chaud', 'réchauffer']
['réchauffer', 'tenir chaud']


['réchauffer']

In [12]:
def get_cross_translate(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_list = get_definitions(word_lang_1, lang_1, target_lang, print_output=True)
    lang_1_processed = preprocess_words(lang_1_to_target_words_list, lemmatizer_func)
    print(lang_1_processed)
    lang_2_to_target_words_list = get_definitions(word_lang_2, lang_2, target_lang, print_output=True)
    lang_2_processed = preprocess_words(lang_2_to_target_words_list, lemmatizer_func)
    print(lang_2_processed)
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_processed, lang_2_processed)
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate("warm", "en", "согревать","ru", "fr", fr_lemmatize)


https://context.reverso.net/перевод/английский-французский/warm
['chaud', 'chaleureux', 'tiède', 'doux', 'bon', 'cordial', 'réchauffer', 'chauffer', 'échauffer', 'se réchauffer', 'tiédir', 'chaleur', 'accueil', 'chaleureusement', 'warm', 'accueillante', 'conviviale']
https://context.reverso.net/перевод/русский-французский/согревать
turn to alternative marks
['réchauffer', 'chaud']


['chaud', 'réchauffer | chauffer | échauffer']

In [69]:
def get_multilang_from_df(df):
    eng_word_list = []
    lang_1_word_list = []
    lang_2_word_list = []
    for df_ind in tqdm(range(len(df))):
        df_element = list(df.iloc[df_ind])
        eng_word = df_element[1]
        loc_1_word = df_element[2]
        print(eng_word, loc_1_word)
        #sense_intersect = get_cross_translate(eng_word, "английский", loc_1_word,"русский", "французский", fr_lemmatize)
        sense_intersect = get_cross_translate_yand(eng_word, "en", loc_1_word,"ru", "fr", fr_lemmatize)
        print("sense_intersect", sense_intersect)
        for sense_intersect_element in sense_intersect:
            eng_word_list.append(eng_word)
            lang_1_word_list.append(loc_1_word)
            lang_2_word_list.append(sense_intersect_element)
            
        #break
        if df_ind % 10 == 0:
            save_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos",
                         str(df_ind) + ".csv")
            data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
            data.to_csv(save_path)
            
    data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
    
    return data

df_din = get_multilang_from_df(df_en_ru_yandex)
df_din.head()

  2%|▏         | 14/676 [00:00<00:04, 133.28it/s]

for для
['pour']
['pour']
sense_intersect ['pour']
for ради
['pour']
['pour']
sense_intersect ['pour']
for в течение
['pour']
['au cours de']
sense_intersect ['no_equality']
for в отношении
['pour']
["à l'égard de", "à l'encontre", 'au regard de', 'au chapitre', 'côté']
sense_intersect ['no_equality']
for для целей
['pour']
[]
sense_intersect ['no_equality']
for ибо
['pour']
['car']
sense_intersect ['no_equality']
for чтобы
['pour']
['pour']
sense_intersect ['pour']
cafe кофейня
['café']
['café', 'cafétéria', 'bar à café']
sense_intersect ['café']
cafe буфет
['café']
['buffet', 'collation']
sense_intersect ['no_equality']
cafe кафе-ресторан
['café']
['café-restaurant']
sense_intersect ['no_equality']
follow следовать
['suivre', 'suivi', 'ensuit']
['suivre', 'consulter', 'respecter', 'dérouler', 'poursuivre', 'se suivre', 'ressortir', "s'ensuivre", 'succéder à', 'professer']
sense_intersect ['suivre']
follow следить
['suivre', 'suivi', 'ensuit']
['surveiller', 'veiller à', 'poursuivre',

  6%|▌         | 38/676 [00:00<00:05, 120.19it/s]

sense_intersect ['no_equality']
warm разминка
['chaud', 'réchauffer']
['échauffement', 'echauffement', 'mise en train']
sense_intersect ['no_equality']
collide сталкиваться
['collision', 'heurter']
['se heurter', 'aborder', "s'entrechoquer", "s'affronter", 'se choquer', 'se croiser', 'se cogner', 'percuter']
sense_intersect ['no_equality']
collide конфликтовать
['collision', 'heurter']
['être en conflit']
sense_intersect ['no_equality']
collide наехать
['collision', 'heurter']
['donner contre']
sense_intersect ['no_equality']
collide соударяться
['collision', 'heurter']
[]
sense_intersect ['no_equality']
collide в конфликт
['collision', 'heurter']
[]
sense_intersect ['no_equality']
break ломать
['briser', 'pause', 'rupture']
['casser', 'déformer', 'déposer']
sense_intersect ['no_equality']
break нарушать
['briser', 'pause', 'rupture']
['violer', 'contrevenir', 'empiéter', 'empiéter sur', 'altérer', 'interrompre', 'entraver', 'bouleverser', 'endommager', 'trahir']
sense_intersect ['no_e

 10%|█         | 69/676 [00:00<00:04, 132.96it/s]

...

 95%|█████████▍| 640/676 [00:04<00:00, 161.83it/s]

['ordre', 'ordre de grandeur', 'procédure', 'modalité', 'système', "c'est ok", 'paix', 'degré', 'o', 'économie', 'tradition']
sense_intersect ['ordre']
order приказ
['ordre', 'commander']
['ordre', 'décret', 'commande', 'département', "mot d'ordre", 'note', 'note de service', 'ordre du jour']
sense_intersect ['ordre']
order заказ
['ordre', 'commander']
['commande', 'ordre', 'bon de commande', 'réservation', 'demande', 'note de commande']
sense_intersect ['ordre']
order орден
['ordre', 'commander']
['ordre', 'décoration']
sense_intersect ['ordre']
order последовательность
['ordre', 'commander']
['séquence', 'succession', 'suivi', 'logique', 'progression', 'suite', 'unité', 'suite logique', 'esprit de suite', 'consécution', 'ligne de continuité', 'rigueur', 'train', 'gradation']
sense_intersect ['no_equality']
order строй
['ordre', 'commander']
['système', 'formation', 'organisation', 'rang', 'édifice', 'modalité']
sense_intersect ['no_equality']
order приказание
['ordre', 'commander']
[

100%|██████████| 676/676 [00:04<00:00, 151.45it/s]

sense_intersect ['traiter']
treat лечить
['traiter']
['traiter', 'guérir']
sense_intersect ['traiter']
treat обрабатывать
['traiter']
['traiter', 'gérer', 'cultiver', 'transformer', 'fabriquer', 'menuiser', 'surfacer', 'ouvrer', 'dresser', 'conditionner', 'dépouiller']
sense_intersect ['traiter']
treat обходиться
['traiter']
['traiter', 'revenir']
sense_intersect ['traiter']
treat угощать
['traiter']
['régaler', 'traiter', 'servir']
sense_intersect ['traiter']
treat побаловать
['traiter']
[]
sense_intersect ['no_equality']
treat третировать
['traiter']
['traiter de haut']
sense_intersect ['no_equality']
treat угощение
['traiter']
['régalade', 'festin', 'consommation']
sense_intersect ['no_equality']
treat удовольствие
['traiter']
['plaisir', 'fun', 'amusement', 'satisfaction', 'bonheur', 'plaisance', 'luxe', 'pied', 'douceur', 'menus plaisirs', 'matière', 'complaisance', 'acompte']
sense_intersect ['no_equality']
treat обработка
['traiter']
['traitement', 'travail', 'façon', 'mise en œ

,eng_word,rus,fr
0,for,для,pour
1,for,ради,pour
2,for,в течение,no_equality
3,for,в отношении,no_equality
4,for,для целей,no_equality


In [71]:
df_din.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 3 columns):
eng_word    683 non-null object
rus         683 non-null object
fr          683 non-null object
dtypes: object(3)
memory usage: 16.1+ KB


In [70]:
df_din.head(20)

,eng_word,rus,fr
0,for,для,pour
1,for,ради,pour
2,for,в течение,no_equality
3,for,в отношении,no_equality
4,for,для целей,no_equality
5,for,ибо,no_equality
6,for,чтобы,pour
7,cafe,кофейня,café
8,cafe,буфет,no_equality
9,cafe,кафе-ресторан,no_equality


In [14]:
#df_din.to_csv("cross_approach.csv")

In [72]:
df_din.to_csv("cross_approach_yandex_vs_yandex.csv")

# Y_LOOkup_azure_cross_aproach

In [22]:
from io import StringIO


In [16]:
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,для,"['for the bank', 'for last month', 'for one ye...","['для банка', 'за последний месяц', 'на один г..."
1,1,for,ради,['for god'],['ради бога']
2,2,for,в течение,[],[]
3,3,for,в отношении,[],[]
4,4,for,для целей,[],[]


In [19]:
def get_stringfied_json(word):
    dt = {'Text': word}
    stringIO = StringIO()
    json.dump(dt, stringIO)
    return stringIO.getvalue()

In [28]:
headers = {
    'Ocp-Apim-Subscription-Key': '7f44a7c95fd34041a2eef780f3f79817',
    'Content-Type': 'application/json',
}

params = (
    ('api-version', '3.0'),
    ('from', 'ru'),
    ('to', 'fr'),
)

In [29]:
for local_word in list(df_en_ru_yandex['local_word']):
    print (local_word)
    data = get_stringfied_json(local_word)
    response = requests.post('https://api.cognitive.microsofttranslator.com/dictionary/lookup', headers=headers, params=params, data=data)
    save_loc = "./lookup/azure_ru_fr/" + local_word + ".json"
    result = json.loads(response.text)
    with open(save_loc, "w") as f:
        json.dump(result,f,indent = 4, ensure_ascii=False)
    break

для


# lang pair not available

# Y_LOOKUP+Y_LOOKUP approach

In [ ]:
def get_definitions(word, from_lang, to_lang, print_output = False):


In [34]:
for local_word in list(df_en_ru_yandex['local_word']):
    print (local_word)
    url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
                  "lang": "ru-fr","text":local_word}
    result = requests.get(url, headers).json()
    save_loc = "./lookup/yandex_ru_fr/" + local_word + ".json"
    #result = json.loads(response.text)
    with open(save_loc, "w") as f:
        json.dump(result,f,indent = 4, ensure_ascii=False)
    

для
ради
в течение
в отношении
для целей
ибо
чтобы
кофейня
буфет
кафе-ресторан
следовать
следить
соблюдать
сопровождать
идти
выполнять
вытекать
далее следовать
повторять
последующий
теплый
сердечный
согревать
погреться
утеплить
тепло
разминка
сталкиваться
конфликтовать
наехать
соударяться
в конфликт
ломать
нарушать
разбивать
разорвать
прорваться
разрушить
вырваться
разбиться
разразиться
пробить
преодолеть
отколоться
вырывать
преломлять
перерыв
разрыв
пролом
каникулы
передышка
тигр
зверь
тигриный
лев
львиный
фон
предпосылка
происхождение
подготовка
история
сведения
фоновый режим
опыт
справочная информация
история вопроса
прошлое
образование
корень
исходная информация
контекст
общие сведения
исток
фоновое изображение
фоновая
фоновый
справочный
второстепенный
писать
записывать
выписывать
вписать
списать
письменный
извиниться
летать
вылететь
влететь
подлететь
отлететь
слетаться
муха
полет
бобр
Бобрик
Бивер
работяга
бородач
бобровый
совпадать
совмещаться
что угодно
ничего
все
исчерпывающе
п

In [36]:
def parse_json_vs_lists(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [62]:
def get_definitions(word, lang):
    if lang == "ru":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_ru_fr/", word + ".json")
    elif lang == "fr":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_en_fr_whole_list/", word + ".json")
    local_word = []
    with open(word_path, "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                local_word.append(translation['text'])
    return local_word

In [61]:
get_definitions("право", "ru")

['droit',
 'pouvoir',
 'licence',
 'acquis',
 'droit de licence',
 'qualité',
 'transmission',
 'droit imprescriptible',
 'ayant cause',
 'droit au brevet',
 'compétence législative',
 'en vérité',
 'vraiment']

In [ ]:
def get_definitions(word, from_lang, to_lang,print_output = False):
    url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
                  "lang": from_lang + '-' to_lang,"text":word}
    ddd = requests.get(url, headers).json()
    ddd